# Computer Systems 2016/17

### Practice 1 - MapReduce with PySpark

#### Here we do somenthing to ensure Spark works

In [1]:
import pyspark
sc = pyspark.SparkContext('local[*]')

# Check that Spark is working
largeRange = sc.parallelize(range(100000))
reduceTest = largeRange.reduce(lambda a, b: a + b)
filterReduceTest = largeRange.filter(lambda x: x % 7 == 0).sum()

print (reduceTest)
print (filterReduceTest)

# If the Spark jobs don't work properly these will raise an AssertionError
assert reduceTest == 4999950000
assert filterReduceTest == 714264285

4999950000
714264285


# Word count

#### First we create a spark Resilient Distributed Dataset (RDD) containing each line from the file.
#### SC is our current SparkContext

In [2]:
lines = sc.textFile('2001 A SPACE ODYSSEY.mht')


#### PySpark provides operations on RDDs to apply transforms produce new RDDs or to return some results.
#### Let's experiment a bit... 

#### First, let's try the map operation. We map each row to its length

In [3]:

lines_length = lines.map( lambda x: len(x))

lines_length.take(10)


[21, 0, 19, 0, 11, 1, 41, 0, 25, 28]

#### Then we apply a reduce operation on the previous result. We reduce the lines length using a sum, obtaining the number of characters in the file.

In [4]:
total_characters = lines_length.reduce(lambda result,x: (result+x))

print ("Total file length is: {} characters".format(total_characters))


Total file length is: 111887 characters


#### To filter out empty lines we can use a filter transformation.

In [5]:
lines_nonempty = lines.filter( lambda x: len(x) > 0 )

print ("Original lines {}, non empty lines {}".format(lines.count(), lines_nonempty.count()))

Original lines 5696, non empty lines 4434


#### Let's remove punctuation marks and transform our data structure made by rows into another one made by words

In [6]:
lines_nonempty = lines_nonempty.map( lambda x: x.replace(',',' ').replace('.',' ').replace('-',' ')\
                                    .replace('!',' ').replace('?',' ').lower())

words = lines_nonempty.flatMap(lambda x: x.split())
words.take(10)

['2001:',
 'a',
 'space',
 'odyssey',
 'screenplay',
 'by',
 'stanley',
 'kubrick',
 'and',
 'arthur']

#### First we replace each original value in the input RDD with a 2-tuple containing the word in the first position and the integer value 1 in the second position.

#### At this point the RDD contains tuples of the form key,value. We create a new RDD containing a tuple for each unique value of key in the input, where the value in the second position of the tuple is created by applying the supplied lambda function to the values with the matching key in the input RDD

#### Here the key will be the word and lambda function will sum up the word counts for each word. The output RDD will consist of a single tuple for each unique word in the data, 

In [7]:
wordcounts = words.map(lambda x: (x, 1)).reduceByKey(lambda x,y:x+y)
wordcounts.take(10)

[('technique', 1),
 ('was', 81),
 ('stirs', 1),
 ('hungry', 2),
 ('combats', 1),
 ('head', 6),
 ('dark', 3),
 ('tools', 1),
 ('b12', 6),
 ('marshall', 1)]

#### Then we map a lambda function to the data which will swap over the first and second values in each tuple, now the word count appears in the first position and the word in the second position.

#### In the end sort the input RDD by the key value

In [8]:
wordcounts = wordcounts.map(lambda x:(x[1],x[0])).sortByKey(ascending=False)
wordcounts.take(10)

[(862, 'the'),
 (396, 'of'),
 (381, 'and'),
 (366, 'to'),
 (270, 'a'),
 (197, 'in'),
 (196, 'it'),
 (176, 'you'),
 (162, 'i'),
 (162, 'is')]

# Finding frequent word bigrams

#### A bigram is pair of successive tokens in some sequence. We will look at building bigrams from the sequences of words in each sentence, and then try to find the most frequently occuring ones.

#### The first problem is that values in each partition of our initial RDD describe lines from the file rather than sentences. Sentences may be split over multiple lines. The glom() RDD method is used to create a single entry for each document containing the list of all lines, we can then join the lines up, then resplit them into sentences using ( . ) as the separator, using flatMap so that every object in our RDD is now a sentence.


In [9]:
lines = sc.textFile('2001 A SPACE ODYSSEY.mht')
lines = lines.map( lambda x: x.replace(',',' ').replace('\t',' ').replace('-',' ')\
                  .replace('!','.').replace('?','.').lower())

sentences = lines.glom() \
            .map(lambda x: " ".join(x)) \
            .flatMap(lambda x: x.split("."))
        
sentences.take(5)

['2001: a space odyssey           screenplay           by         stanley kubrick and arthur c',
 ' clark           hawk films ltd',
 '           c/o',
 ' m g m studios           boreham wood           herts',
 '   title         part i          africa          3 000 000 years ago  a1 views of african drylands   drought  the remorseless drought had lasted now for ten million years  and would not end for another million']

#### Now we have isolated each sentence we can split it into a list of words and extract the word bigrams from it. Our new RDD contains tuples containing the word bigram (itself a tuple containing the first and second word) as the first value and the number 1 as the second value.

In [10]:
bigrams = sentences.map(lambda x:x.split()) \
    .flatMap(lambda x: [((x[i],x[i+1]),1) for i in range(0,len(x)-1)])

bigrams.take(5)

[(('2001:', 'a'), 1),
 (('a', 'space'), 1),
 (('space', 'odyssey'), 1),
 (('odyssey', 'screenplay'), 1),
 (('screenplay', 'by'), 1)]

#### Finally we can apply the same reduceByKey and sort steps that we used in the wordcount example, to count up the bigrams and sort them in order of descending frequency. In reduceByKey the key is not an individual word but a bigram.

In [11]:
freq_bigrams = bigrams.reduceByKey(lambda x,y:x+y) \
    .map(lambda x:(x[1],x[0])) \
    .sortByKey(ascending=False)
    
freq_bigrams.take(10)

[(90, ('of', 'the')),
 (49, ('in', 'the')),
 (41, ('to', 'the')),
 (32, ('on', 'the')),
 (32, ('at', 'the')),
 (31, ('pod', 'bay')),
 (26, ('and', 'the')),
 (26, ('it', 'is')),
 (25, ('to', 'be')),
 (24, ('we', 'see'))]

# Simpe parallelization with custom functions

### Pi estimation sequential

In [13]:
import random
import time

def inside(p):
    x, y = random.random(), random.random()
    return x*x + y*y < 1

NUM_SAMPLES = 50000000

starting_time = time.time()

count = 0

for index in range(0, NUM_SAMPLES):
    if inside(index):
        count +=1

    
print ("Pi is roughly %f" % (4.0 * count / NUM_SAMPLES))
print ("Computed in {} seconds".format(time.time() - starting_time))

Pi is roughly 3.141796
Computed in 20.626342058181763 seconds


### Pi estimation parallel

In [15]:
import random
import time

def inside(p):
    x, y = random.random(), random.random()
    return x*x + y*y < 1

NUM_SAMPLES = 50000000

starting_time = time.time()

count = sc.parallelize(range(0, NUM_SAMPLES)) \
             .filter(inside).count()
    
print ("Pi is roughly %f" % (4.0 * count / NUM_SAMPLES))
print ("Computed in {} seconds".format(time.time() - starting_time))

Pi is roughly 3.141702
Computed in 6.212697744369507 seconds
